## Introduction
#### Over time, pandas has developed depth of features for power users.
#### To learn these advanced features will help you deepen your expertise as pandas user.

## Categorical Data
#### We can use the pandas categorical data type to achieve better performance and memory use.
#### We can also use it in statistics and machine learning applications.

### Background and Motivation
#### Sometimes a column in a table may contain repested instances of smaller set of distinct values.
#### We know about methods like 'unique' (allows us to extract distinct values from array) and 'value_counts' (allows us to compute their frequencies). 

In [4]:
import numpy as np
import pandas as pd

values = pd.Series(['apple', 'orange', 'apple',
                   'apple'] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

#### Data Systems (used for data warehousing, statistical computing, etc.) have developed specialized approaches to represent data with repeated values.
#### In Data Warehousing, a best practice is to use 'dimension tables' having distinct values. Primary observations are stored as integer keys referncing teh dimension table.

In [8]:
values = pd.Series([0,1,0,0] * 2)
dim = pd.Series(['apple', 'orange'])

values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

#### We can use the 'take' method to restore the original Series of strings in above example.
#### This representation as integers is called 'categorical' or 'dictionary-encoded' representation.
#### The array of distinct values can be called 'categories', 'dictionary' or 'levels' of data.
#### The integer values that refence the categories are called 'category codes' or simply 'codes'.

In [11]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

#### Categorical representation yields significant performance improvements when doing analytics.
#### We can also perform transformations on these categories without modifying original data. eg - renaming categories, appending new category without changing position or order of older category, etc.

### Categorical Type in pandas
#### pandas has special 'Categorical' type for holding data that uses integer based categorical representation or 'encoding'.
#### We can convert an array of any data type into categorical by using "astype('category')" on the array. This results in an instance of 'pandas.Categorical'.
#### the Categorical object has 'categorical' and 'codes' atributes.

In [14]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)},
                 columns = ['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,6,3.771870
1,1,orange,10,0.033614
2,2,apple,12,2.829285
3,3,apple,3,2.143480
4,4,apple,3,3.191547
5,5,orange,9,1.303247
6,6,apple,9,0.865479
7,7,apple,8,0.617000


In [15]:
fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [16]:
c = fruit_cat.values

type(c)

pandas.core.arrays.categorical.Categorical

In [17]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [18]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

#### You can convert a DataFramecolumn to categorical by assigning it the converted result.
#### You can convert other types of Python Sequences into categorical data type as well.
#### In case you have received the categorical encoded data (i.e. category and code) from another source, you can use 'from_codes' to create a categorical data out of it.

In [20]:
df['fruit'] = df['fruit'].astype('category')

df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [22]:
# Creating category out of Python list.
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [23]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

#### Categorical data have no specific ordering unless specifically defined.
#### So the 'categories' array may be in different order based on ordering of input data.
#### When using 'from_codes' you have the option to indeicate that there is a meaningful order in the categories.
#### When looking at the data in output, you can also see the ordering of the categories.
#### An unordered categorical instance can be made ordered with 'as_ordered'.

In [25]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered=True)
# Shows that 'foo' precedes 'bar' in ordering and so on.
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [26]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### Computations with Categoricals
#### Using Categorical data instead of the original data (eg - array of strings) behaves the same way.
#### Some parts of pandas (eg - groupby) work better with categoricals. Some other functions utilize the ordered flag as well.
#### Some functions (eg - pandas.qcut binning function) returns categorical data. The labels are the quartiles itself. We can change the labels by passing 'labels' argument.

In [29]:
np.random.seed(12345)
draws = np.random.randn(1000)

draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [30]:
bins = pd.qcut(draws, 4)

bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [31]:
# Using labels to classify the quartiles
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [32]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

#### The labelled categorical does not contain the any information about the bin edges in the data. We can use 'groupby' to extract the summary statistics.
#### The 'quartile' column in the example 'results' retains original categorical information.

In [34]:
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [35]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### Better Performance with Categories
#### Converting to categorical data can yield substantial eprformance gains in analytical processes.
#### Categorical dsata uses significantly less memory as compared to arrays of strings.
#### Obviously the conversion is not free, but is just a one-time cost.
#### Different operations (like GroupBy) can be significantly faster with categoricals because underlying algorithms use integer-based codes instead of string arrays.

In [37]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

categories = labels.astype('category')

In [38]:
labels.memory_usage()

80000080

In [39]:
categories.memory_usage()

10000272

In [40]:
%time _ = labels.astype('category')

Wall time: 368 ms


### Categorical Methods
#### Series having categorical data has several special methods. It provides convinient access to categories and codes.
#### The special attribute 'cat' provides access to categorical methods.

In [42]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [43]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [44]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

#### We can use 'set_categories' if we know that there are more categories than we currently see in the data and we want to include other categories as well.
#### Even if there is no data that corresponds to the extra categories, these categories will be reflected in operations that use them. eg - value_counts.
#### For large datasets, categoricals is very convinient for memory savings and better performance.
#### After filtering a large dataset, you may not have all the categories having any data. To trim such categories, use 'remove_unused_categories'.

In [46]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [48]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [49]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [50]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [51]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

#### Creating Dummy Variables for Modeling
#### In statistics and machine learning, to transform categorical data into 'dummy variables' is called 'one-hot encoding'.
#### This involves creating DataFrame with a column for each distinct category. The column contains 1s for occurences of a category and 0s otherwise.
#### The function 'pandas.get_dummies' converts 1-dimensional categorical data into a DataFrame having dummy variable.

In [54]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [55]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## Advanced GroupBy Use
#### There are a few advanced techniques in groupby that can be useful in data analysis.

### Group Transforms and "Unwrapped" GroupBys
#### Just like the 'apply' method for grouped operations, we also have 'transform', which is similar but imposes more constraints on kind of function you can use.
#### The restrictions on the function are:
####     1. It can produce scalar value to be broadcast to the shape of the group.
####     2. It can produce an object of the same shape as input group.
####     3. It must not mutate its input.

In [58]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                  'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


#### Let's group the above DataFrame by the key using the mean of each key as value.
#### Now if we want to produce a Series with same length as the above Dataframe's 'value' column, but with values replaced by the average grouped by key.
#### To achieve this we can pass the function 'lambda x: x.mean()' to 'transform'.

In [60]:
g = df.groupby('key').value

g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [61]:
# Replaced value with mean of the key group
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

#### For built-in aggregation methods, we can use a string alias instead of lambda. eg - 'mean' for getting mean in the values.
#### Transform works with functions that return Series but the result should be of same size as the input.
#### We can perform more complicated operations by passing a more complicated lambda function to transform. eg - ranking in descending order for each group.

In [63]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [64]:
# As long as result is of same length as input, transform works
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [66]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

#### For a group transformation composing simple aggregations, 'transform' and 'apply' lead to equivalent results.
#### Built-in aggregation functions (eg - mean, sum, etc) are much faster than a general 'apply' function.
#### They also have a 'fast past' when used with 'transform', which allows us to perform 'unwrapped' group operation.
#### An unwrapped group operation involves multiple group aggregations, but there is overall benefit of vectorized operations.

In [68]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [69]:
# Equivalent result for both transform and apply
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [70]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [72]:
# Unwrapped group aggregations are faster because of inbuilt aggregations

normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Grouped Time Resampling
#### For time series, 'resample' is semantically a group operation based on time intervalization.
#### We usually use the time as index and then resample, grouping based on index (time).

In [74]:
N = 15

times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                  'value': np.arange(N)})

df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [75]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


#### But if a DataFrame has a time series and a key column, then grouping cannot be done based on time alone.
#### To do a resample for each value of the 'key', we use the 'pandas.TimeGrouper' (now 'pandas.Grouper') object.
#### We set the time as index, then group by key and time and then aggregate.
#### One constraint using TimeGrouper is that the time must be the index of the Series or DataFrame.

In [77]:
df2 = pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})

df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [81]:
time_key = pd.TimeGrouper('5min')

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [82]:
resampled = (df2.set_index('time')
            .groupby(['key', time_key])
            .sum())

resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

### Techniques for Method Chaining
#### When applying a sequence of transformations to a dataset, you may create many temporary variables which you will never use in your analysis.
#### There are ways of avoiding this. The 'DataFrame.assign' method is a 'functional' alternative to column assignments. eg - df[k] = v.
#### Rather than modifying object in-place, it returns a new DataFrame with indicated modifications.

In [88]:
N = 15
df = pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})
df2 = df[df['value'] < 0]
df2['value'] = df2['value'] - df2['value'].mean()
result = df2.groupby('key').std()

In [90]:
# Non-functional way
df2 = df.copy()
df2['key'] = 'x'

# Functional way
df2 = df.assign(k='x')

#### Assigning in-place may execute faster, but 'assign' enables easier method chaining.
#### While performing 'method chaining', keep in mind that you may need to refer to temporary objects. To help, 'assign' and many other pandas functions accept function-like arguments called 'callables'.
#### The function we need to pass is kept in [], which is then bounded to the object at that stage.
#### The use of method chaining and callables allows us to write multiple functions in a single line of code.
#### But doing so is a matter of taste. Splitting up expression into multiple steps is more readable.

In [92]:
# Method chaining with assign
result = (df2.assign(col1_demeaned = df2.value - df2.value.mean())
         .groupby('key')
         .col1_demeaned.std())

In [93]:
# Using Callable in first example
df = (pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})
     [lambda x: x['value'] < 0])

In [94]:
# Writing single line expression with callable and method chaining
result = (pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})
     [lambda x: x['value'] < 0]
         .assign(col1_demeaned = df2.value - df2.value.mean())
          .groupby('key')
         .col1_demeaned.std())

### The pipe Method
#### You can achieve a lot with pandas built-in functions with method chaining and callables.
#### But sometimes you need to use your own functions or those from third-party libraries. This is where 'pipe' method comes in.
#### When using functions that accept and return Series and DataFrame from previous operation, you can rewrite in a single line using calls to 'pipe'.

In [96]:
# Series of functions using output from previous function as one of the input
'''
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
'''

'\na = f(df, arg1=v1)\nb = g(a, v2, arg3=v3)\nc = h(b, arg4=v4)\n'

In [97]:
# Same operation using 'pipe'
'''
result = (df.pipe(f, arg1=v1)
         .pipe(g, v2, arg3=v3)
         .pipe(h, arg4=v4))
'''

'\nresult = (df.pipe(f, arg1=v1)\n         .pipe(g, v2, arg3=v3)\n         .pipe(h, arg4=v4))\n'

#### Although the statements 'f(df)' and 'df.pipe(f)' are equivalent, pipe makes chained invocation easier.
#### A potentially useful pattern for pipe is to generalize a sequence of operations into resuable functions. eg - demeaning multiple columns.

In [99]:
# A simple demeaning (subtracting mean from a value) operation
'''
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
'''

"\ng = df.groupby(['key1', 'key2'])\ndf['col1'] = df['col1'] - g.transform('mean')\n"

In [100]:
# Demeaning function
'''
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
'''

"\ndef group_demean(df, by, cols):\n    result = df.copy()\n    g = df.groupby(by)\n    for c in cols:\n        result[c] = df[c] - g[c].transform('mean')\n    return result\n"

In [101]:
# Using demean function with pipe
'''
result = (df[df.col1 < 0]
         .pipe(group_demean, ['key1', 'key2'], ['col1']))
'''

"\nresult = (df[df.col1 < 0]\n         .pipe(group_demean, ['key1', 'key2'], ['col1']))\n"